In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [2]:
# random.sample(range(0, len(image_names)), 5)

In [2]:
# image_names = get_png_names("../pooling/data/MEX2/")#[100:105]
image_names = get_png_names("../pooling/data/MEX2/")
# image_indices = random.sample(range(0, len(image_names)), 1000)
# image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../pooling/data/migration_data.json" , image_names)

1779 municipalities.


In [3]:
# for i in image_names:
#     print(load_inputs(i).shape)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .0001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

fc_model = fc_net(resnet = resnet18).to(device)
fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr = .01)

butler = geozoom_handler(attn_model, 
                         fc_model, device, 
                         criterion, 
                         attn_optimizer, 
                         fc_optimizer, 
                         num_thresholds = 10,
                         reduction_percent = .90,
                         convergence_dims = (358, 284),
                         plot = False, 
                         v = False)

In [5]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [6]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [7]:
# for i,o in val_dl:
#     print(load_inputs(i[0]).shape, o)

In [ ]:
butler.train_attn_model(train_dl, val_dl)

Epoch:  0
  Training Loss:  1681.596755298813
  Validation Loss:  1965.0938637042314
Loss thresholds for training:  [0.0, 196.50938637042313, 393.01877274084626, 589.5281591112694, 786.0375454816925, 982.5469318521157, 1179.0563182225387, 1375.565704592962, 1572.075090963385, 1768.584477333808]
Starting from threshold:  9  with value:  1768.584477333808


Epoch:  1
  Training Loss:  1632.5385412486112
  Validation Loss:  1917.9902844000399


Epoch:  2
  Training Loss:  1605.5330144422794
  Validation Loss:  1893.74773142311


Epoch:  3
  Training Loss:  1589.1283436053918
  Validation Loss:  1872.8428049087524


Epoch:  4
  Training Loss:  1577.7314240903415
  Validation Loss:  1864.9426038292017


Epoch:  5
  Training Loss:  1568.0445639870384
  Validation Loss:  1862.4357418317472


Epoch:  6
  Training Loss:  1558.6766069978894
  Validation Loss:  1813.8224839842721


Epoch:  7
  Training Loss:  1547.414952901109
  Validation Loss:  1831.2996347620247


Epoch:  8
  Training Loss:  1

/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")




Epoch:  84
  Training Loss:  1429.1438399138804
  Validation Loss:  1977.912740793121


Epoch:  85
  Training Loss:  1443.609296432252
  Validation Loss:  2354.3551231984343


Epoch:  86
  Training Loss:  1435.6123383226188
  Validation Loss:  2284.4977085027804


Epoch:  87
  Training Loss:  1427.8373026253469
  Validation Loss:  2230.58977242802


Epoch:  88
  Training Loss:  1424.711346326862
  Validation Loss:  2184.667084768917


Epoch:  89
  Training Loss:  1426.4470689339178
  Validation Loss:  1911.9869512065072


Epoch:  90
  Training Loss:  1424.2231230659806
  Validation Loss:  2041.0146849557254


Epoch:  91
  Training Loss:  1408.6560054732577
  Validation Loss:  2131.502946082126


Epoch:  92
  Training Loss:  1419.9028056179573
  Validation Loss:  3680.5739815101197


Epoch:  93
  Training Loss:  1405.2243835106003
  Validation Loss:  2258.1718199011984


Epoch:  94
  Training Loss:  1397.2349324105717
  Validation Loss:  2224.146176927545


Epoch:  95
  Training Loss: